In [43]:
import pandas as pd
import numpy as np
import json
from datetime import date
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [44]:
artist_df = pd.read_csv('./data/artists.csv')
tracks_df = pd.read_csv('./data/tracks.csv')
with open("./data/dict_artists.json", encoding='utf-8', errors='ignore') as json_data:
     data = json.load(json_data, strict=False)

In [45]:
artist_df.drop(['id'], axis=1, inplace=True)

In [46]:
artist_df = artist_df[artist_df['genres'] != '[]']
artist_df = artist_df.sort_values(by=['popularity'], ascending=False)
l = len(artist_df)/15
artist_df = artist_df[:round(l)]
artist_df = artist_df.reset_index(drop=True)

In [47]:
artist_df

,followers,genres,name,popularity
0,44606973.0,"['canadian pop', 'pop', 'post-teen pop']",Justin Bieber,100
1,54416812.0,"['canadian hip hop', 'canadian pop', 'hip hop'...",Drake,98
2,32244734.0,"['latin', 'reggaeton', 'trap latino']",Bad Bunny,98
3,38869193.0,"['pop', 'post-teen pop']",Taylor Swift,98
4,16996777.0,"['chicago rap', 'melodic rap']",Juice WRLD,96
...,...,...,...,...
19903,2394.0,['russian edm'],Denis First,50
19904,1028316.0,"['sertanejo', 'sertanejo pop', 'sertanejo univ...",Jefferson Moraes,50
19905,33168.0,"['danish hip hop', 'danish pop']",Bro,50
19906,15823.0,"['bass house', 'deep groove house', 'electro h...",Marten Hørger,50


In [49]:
model = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = model.fit_transform(artist_df['genres'])

In [50]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix) 

In [51]:
indices = pd.Series(artist_df.index, index=artist_df['name'])

In [52]:
# names = artist_df['name']
# def artist_recommend(name):
#     idx = indices[name]
#     sim_scores = list(enumerate(cosine_similarities[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:31]
#     artist_indices = [i[0] for i in sim_scores]
#     return names[artist_indices].values

In [53]:
# new = artist_recommend("Queen")

In [54]:
#Alternate Method

In [56]:
for idx, row in artist_df.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], artist_df['name'][i]) for i in similar_indices] 
    results[row['name']] = similar_items[1:]

In [106]:
def _recommend(item_id, num):
    recs = results[item_id][:num]   
    preds = {}
    for pair in recs:
        preds[pair[1]] = pair[0]
    return preds

In [109]:
_recommend('Drake', 5)

{'Tory Lanez': 0.5683541991268033,
 'NAV': 0.5676320403065631,
 'NF': 0.49303744366086255,
 'Ali Gatie': 0.4875486352986397,
 'Kardinal Offishall': 0.45975629790111777}

In [121]:
import pickle 
with open('cos_sim_results','wb') as f: pickle.dump(results, f)

In [129]:
def get_similar_artists_multiple(artists, num=10):
    dict_similar = {}
    for artist, weight in artists.items():
        dict_similar[artist] = recommend(artist, num)
    artists_all = []
    for artist, similar_artists in dict_similar.items():
        artists_all.append(list(similar_artists.keys()))
    artists_unique = np.unique(artists_all).tolist()
    artists_dict = {artist: 0 for artist in artists_unique}
    for artist, similar_artists in dict_similar.items():
        for similar_artist, score in similar_artists.items():
            artists_dict[similar_artist] += artists[artist] * score
    return list({k: v for k, v in sorted(artists_dict.items(), key=lambda item: item[1], reverse=True) if k not in artists}.keys())[0:num]

In [131]:
get_similar_artists_multiple({'Drake':10}, 10)

['Tory Lanez',
 'NAV',
 'NF',
 'Ali Gatie',
 'Kardinal Offishall',
 'PARTYNEXTDOOR',
 '404vincent',
 'Duvy',
 'Murda Beatz',
 'Wasiu']